In [1]:
import pandas as pd
from adapt.instance_based import TrAdaBoostR2
from sklearn.ensemble import HistGradientBoostingRegressor as hgb
from sklearn.metrics import mean_absolute_error

In [2]:
#Source=S7
source = 'S7'

data_s = pd.read_csv(f'data/data_s_{source}.csv')
data_s = data_s.drop(columns=['Name', 'ID'])

data_t_lab = pd.read_csv('data/data_t_lab_iso.csv')
data_t_lab = data_t_lab.drop(columns=['Name', 'ID'])
data_t_unlab = pd.read_csv('data/data_t_unlab_iso.csv')
ID_t_unlab = data_t_unlab[['Name', 'ID']]
data_t_unlab = data_t_unlab.drop(columns=['Name', 'ID'])

ys = pd.DataFrame(data_s['Yield'],columns=['Yield'])
Xs = data_s.drop(columns=['Yield'])
yt_lab = pd.DataFrame(data_t_lab['Yield'],columns=['Yield'])
Xt_lab = data_t_lab.drop(columns=['Yield'])
Xt_unlab = data_t_unlab

model = TrAdaBoostR2(hgb(), n_estimators=23, Xt=Xt_lab, yt=yt_lab, random_state=0, verbose=0)
model.fit(Xs, ys)
y_pred = model.predict(Xt_unlab)
y_pred = pd.DataFrame(y_pred, columns=['Pred_yield'])
prediction = pd.concat([ID_t_unlab, y_pred], axis=1, join='inner')
prediction = prediction.sort_values('Pred_yield', ascending=False)
prediction.to_csv(f'result/prediction_iso_{source}.csv', index=False)
prediction.head()

,Name,ID,Pred_yield
5,4DPAIPN,OPS7,89.806493
9,4BrDPAIPN,OPS12,88.986394
2,4MeCzIPN,OPS3,85.314904
8,4ClDPAIPN,OPS11,84.492548
0,4CzIPN,OPS1,82.544285


In [3]:
exp = {'Exp_yield': [92, 95, 72, 96, 91]}
exp = pd.DataFrame(exp)
pred = pd.read_csv(f'result/prediction_iso_{source}.csv')
df = pd.concat([pred, exp], axis=1, join='inner')
mae = mean_absolute_error(df['Exp_yield'], df['Pred_yield'].round().astype(int))
print(f'MAE when using {source}:', round(mae, 1))
df

MAE when using S7: 8.2


,Name,ID,Pred_yield,Exp_yield
0,4DPAIPN,OPS7,89.806493,92
1,4BrDPAIPN,OPS12,88.986394,95
2,4MeCzIPN,OPS3,85.314904,72
3,4ClDPAIPN,OPS11,84.492548,96
4,4CzIPN,OPS1,82.544285,91
